In [53]:
import emoji
import pandas as pd

In [54]:
SEED = 42

In [55]:
prompt_dataset = pd.read_csv('translation_dataset.csv')

prompt_dataset.rename(columns={
    'PROMPT': 'prompt',
    'SCINARIO': 'requirement'
    },inplace=True)

In [56]:
remove_vocab = ['아이:', '트럼프', '문재인', '대통령', '해볼래!', '알려줘', 'Model:', '모델:']

def has_remove_vocab(s):
    for v in remove_vocab:
        if v in s:
            return True
    
    return False

In [57]:
drop_list = []

for i, row in prompt_dataset.iterrows():
    if emoji.emoji_count(row['prompt']) + emoji.emoji_count(row['requirement']) > 0: # has emoji
        drop_list.append(i)
    
    elif len(row['requirement'].split('\n')) > 1: # has dialogue
        drop_list.append(i)
        
    elif has_remove_vocab(row['requirement']): # has remove vocab
        drop_list.append(i)

In [58]:
prompt_dataset.drop(drop_list, inplace=True)

In [59]:
len(prompt_dataset)

7550

In [60]:
prompt_dataset.head()

,Unnamed: 0,prompt,requirement
0,0,"Mind Manifesting poseidon in ocean, holding tr...",바다에서 포세이돈이 살고 있는 모습을 그려봐. 트라이던트를 들고 있고 상어들이 있는...
1,0,Diagram Make The Starling Night paintings real...,별이 밤에 나오는 그림을 그려봐. 나무 속에서 그려도 돼? 그리고 그림은 4K화질이...
2,0,"Magical Girl cute kitten wearing suit, teachin...",귀여운 고양이가 교실에서 가르치는 그림을 그려봐. 밝은 빛이 비추고 배경은 맑게 해...
3,0,"Frank Martin sleeve tattoo , wave, style of Ho...","프랭크 마틴의 팔 터틀넥 타투를 그려봐. 파도와 호쿠사이 스타일, 벚꽃, 지는 해,..."
5,0,Otaku wide shot of a huge red giant star in sp...,우주에서 큰 빨간 거성의 넓은 경치를 그려봐. 멀리에는 작은 파괴된 우주선의 실루엣...


In [61]:
from datasets import Dataset

In [62]:
prompt_dataset = Dataset.from_pandas(prompt_dataset)
# Split the dataset into train and test sets (90% train, 10% test)
prompt_dataset = prompt_dataset.train_test_split(test_size=0.1, seed=SEED)

# Split the train set further into train and validation sets (90% train, 10% validation)
prompt_dataset['train'] = prompt_dataset['train'].train_test_split(test_size=0.1, seed=SEED)

# Access the resulting datasets
train_dataset = prompt_dataset['train']['train'].to_pandas()
valid_dataset = prompt_dataset['train']['test'].to_pandas()
test_dataset = prompt_dataset['test'].to_pandas()

In [63]:
train_dataset.drop(['Unnamed: 0', '__index_level_0__'], axis=1, inplace=True)
valid_dataset.drop(['Unnamed: 0', '__index_level_0__'], axis=1, inplace=True)
test_dataset.drop(['Unnamed: 0', '__index_level_0__'], axis=1, inplace=True)

In [64]:
train_dataset.head()

,prompt,requirement
0,Honored producing 10 frames of a sequence with...,"놀라움을 표현하는 사람, 믿을 수 없어서 입을 크게 벌리고 손으로 가린 사람, 카메..."
1,Concept marcus aurelius in roman cathedral fig...,로마 대성당에서 마르쿠스 아우렐리우스가 사자와 싸우는 그림을 그려봐. 역사적인 현실...
2,"Draping Deadpool anime style, dramatic cinemat...",데드풀을 애니메이션 스타일로 그려봐. 장관적이고 시네마틱한 도시 환경에 드레이핑되어...
3,Silky A synesthetic experience where sounds ar...,소리가 눈에 보이는 그림을 그려봐. 가장 높은 해상도와 디테일로.
4,Bulk Stickers car accident 4k pro photagraphy ...,차 사고 그림을 그려봐. 그리고 사진은 4K로 찍어줘. 사고는 심각하지 않아서 부스...


In [65]:
def make_accumulate_dataset(df, file_name):
    templates = []

    for i, row in df.iterrows():
        template_input = row['requirement']
        instruction = '아이의 요구사항을 그림 생성을 위한 프롬프트를 번역하세요.'
        
        PROMPT_TEMPLATE = [
            "Below is an instruction that describes a task, paired with an input that provides further context.\n",
            "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n",
            "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n",
            f"### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{template_input}\n\n### Response(응답):"
            ]
        
        templates.append(''.join(PROMPT_TEMPLATE))

    df['template'] = templates
    df.to_csv(file_name, index=False)
    
    print(f'file name: {file_name}, len: {len(df)}')

In [66]:
make_accumulate_dataset(train_dataset, 'train.csv')
make_accumulate_dataset(valid_dataset, 'valid.csv')
make_accumulate_dataset(test_dataset, 'test.csv')

file name: train.csv, len: 6115
file name: valid.csv, len: 680
file name: test.csv, len: 755
